## 3(a) Linear Programming Model

The variables for the problem are defined as follows, <br>
$x_0$ - Amount of 2 year loan borrowed in $Q_1$ <br>
$x_{3i-2}$ - Amount of 6 month loan borrowed in $Q_i$ <br>
$x_{3i-1}$ - Amount of 3 month loan borrowed in $Q_i$ <br>
$x_{3i}$ - Amount of cash surplus available after $Q_i$ <br>

There are 25 variables and 8 equality constraints for the given problem. We assume that the loans are repaid by equated montly instalments (EMI). Using the formula $P*R*\frac{(1+R)^N}{(1+R)^N-1}$, we calculate the EMI for each type loan. The standard form for this LP is given below, <br>

$$\min -1.005x_{24}$$

$$0.86974x_0 + 0.489448x_1 - 0.016709x_2 -x_3 = 100$$
$$-0.13026x_0 - 0.510552x_1 + 1.005x_3 + 0.489448x_4 - 0.016709x_5 -x_6 = 500$$
$$-0.13026x_0 - 0.510552x_4 + 1.005x_6 + 0.489448x_7 - 0.016709x_8 -x_9 = 100$$
$$-0.13026x_0 - 0.510552x_7 + 1.005x_9 + 0.489448x_{10} - 0.016709x_{11} -x_{12} = -600$$
$$-0.13026x_0 - 0.510552x_{10} + 1.005x_{12} + 0.489448x_{13} - 0.016709x_{14} -x_{15} = -500$$
$$-0.13026x_0 - 0.510552x_{13} + 1.005x_{15} + 0.489448x_{16} - 0.016709x_{17} -x_{18} = 200$$
$$-0.13026x_0 - 0.510552x_{16} + 1.005x_{18} + 0.489448x_{19} - 0.016709x_{20} -x_{21} = 600$$
$$-0.13026x_0 - 0.510552x_{19} + 1.005x_{21} + 0.489448x_{22} - 0.016709x_{23} -x_{24} = -900$$

$$x_i \geq 0$$

## 3(b) Revised Simplex Algorithm

In [1]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.optimize import linprog
import time

In [5]:
def revised_simplex(c, A, b, max_iterations=100, init_basic=None):
    """
    Implementation of Revised Simplex Algorithm
    """
    i=1    
    m, n = A.shape
    II = set([j for j in range(n)])
    if init_basic is None:
        BB = set([j for j in range(n-m, n)])
    else:
        BB = init_basic
    NN = II.difference(BB)   
    x = np.zeros(n)
    while i < max_iterations:
        sBB = sorted(BB)
        sNN = sorted(NN)
        B = A[:, sBB]
        N = A[:, sNN]
        cB = c[sBB]
        cN = c[sNN]
        BI = np.linalg.pinv(B)
        # xB = np.linalg.solve(B, b)
        xB = np.matmul(BI, b)
        x[sBB] = xB
        # l = np.linalg.solve(B.transpose(), cB)
        l = np.matmul(BI.transpose(), cB)
        sN = cN - np.matmul(N.transpose(), l)
        if min(sN) >= 0:
            return x, i, True
        else:
            q = np.argmin(sN)
            Q = A[:, sNN[q]]
            # d = np.linalg.solve(B, Q)
            d = np.matmul(BI, Q)
            if max(d) <= 0:
                return x, i, False
            else:
                p = np.argmin(xB/d)
                BB = BB.difference({sBB[p]}).union({sNN[q]})
                NN = II.difference(BB)
        i+=1
    sBB = sorted(BB)
    B = A[:, sBB]
    BI = np.linalg.pinv(B)
    xB = np.matmul(BI, b)
    x[sBB] = xB
    return x, i, True
            
    

## 3(c) LP Solution

In [6]:
# LP Problem Input
A = np.array([[1, 1, 1, 0], [2, 0.5, 0, 1]])
b = np.array([5, 8]).transpose()
c = np.array([-3, -2, 0, 0]).transpose()

# Call Revised Simplex
x, i, found = revised_simplex(c, A, b, 20)

# Print Results
print(x, i, found)

[3.66666667 1.33333333 1.         8.        ] 3 True


In [9]:
# LP Problem Input
A = np.array([[0.86974, 0.489448, -0.016709, -1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
              [-0.13026, -0.510552, 0, 1.005, 0.489448, -0.016709, -1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
              [-0.13026, 0, 0, 0, -0.510552, 0, 1.005, 0.489448, -0.016709, -1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
              [-0.13026, 0, 0, 0, 0, 0, 0, -0.510552, 0, 1.005, 0.489448, -0.016709, -1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
              [-0.13026, 0, 0, 0, 0, 0, 0, 0, 0, 0, -0.510552, 0, 1.005, 0.489448, -0.016709, -1, 0, 0, 0, 0, 0, 0, 0, 0, 0],
              [-0.13026, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, -0.510552, 0, 1.005, 0.489448, -0.016709, -1, 0, 0, 0, 0, 0, 0],
              [-0.13026, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, -0.510552, 0, 1.005, 0.489448, -0.016709, -1, 0, 0, 0],
              [-0.13026, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, -0.510552, 0, 1.005, 0.489448, -0.016709, -1]])
b = np.array([100, 500, 100, -600, -500, 200, 600, -900]).transpose()
c = np.array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0.510552, 0, -1.005]).transpose()

# Call Revised Simplex
x, i, found = revised_simplex(c, A, b, 200, {2, 5, 8, 11, 14, 17, 20, 23})

# Print Results
print(x, i, found)

[ 0.00000000e+00  0.00000000e+00 -5.98479861e+03  0.00000000e+00
  0.00000000e+00 -2.99239931e+04  0.00000000e+00  0.00000000e+00
 -5.98479861e+03  0.00000000e+00  0.00000000e+00  3.59087917e+04
  0.00000000e+00  0.00000000e+00  2.99239931e+04  3.49355187e+02
  0.00000000e+00 -1.51704198e+01 -1.99746517e+02  0.00000000e+00
 -1.17739853e+01 -8.00548519e+02  0.00000000e+00  1.59440783e+00
  9.54220979e+01] 200 True


C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:38: RuntimeWarning: divide by zero encountered in true_divide


## 3(d) Other Packages

In [44]:
# Call Scipy LP Solver
result = linprog(c, A_eq=A, b_eq=b, method='Simplex')

# Print Results
print(result)

     con: array([ 0.00000000e+00, -5.68434189e-14, -5.68434189e-14,  0.00000000e+00,
        1.13686838e-13, -2.27373675e-13,  0.00000000e+00,  0.00000000e+00])
     fun: -462.51913808223037
 message: 'Optimization terminated successfully.'
     nit: 8
   slack: array([], dtype=float64)
  status: 0
 success: True
       x: array([1134.74844104,    0.        ,    0.        ,  889.9420564 ,
          0.        ,    0.        ,  246.57943476,    0.        ,
          0.        ,    0.        ,    0.        ,    0.        ,
        452.18766807,    0.        ,    0.        ,  806.63627448,
          0.        ,    0.        ,  462.85712392,  577.46874517,
          0.        ,    0.        ,    0.        ,    0.        ,
        460.21804784])
